<center><a href="https://www.pieriantraining.com/" ><img src="../PTCenteredPurple.png" alt="Pierian Training Logo" /></a></center>


# Assistant with Code Interpreter

- Executed in colab

In [2]:
from openai import OpenAI
import os
from google.colab import userdata


OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

client = OpenAI()

### Can LLMs do math?

If LLMs are just predicting next most likely tokens, can they actually solve math problems? Or would they hallucinate the answer?

In [3]:
answer = 123456 * 456789
answer

56393342784

In [5]:
response = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [
    {'role':'system','content':'You help solve math problems'},
    {'role':'user','content':'What is 123456 times 456789?'}
    ]

)
print(response.choices[0].message.content)

123456 times 456789 is equal to 56,419,083,584


- Direct LLM gives wrong answer

In [8]:
response = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [
    {'role':'system','content':'You help solve math problems'},
    {'role':'user','content':'What is 123456 * 456789?'}
    ]

)

print(response.choices[0].message.content)

123456 * 456789 = 56,298,699,984


- Direct LLM gives wrong answer

### Asking for code instead, and then running that code!
- Here if we use python code string inside eval it gives ans

In [13]:
response = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [
    {'role':'system','content':'You convert word problems into Python code. Only return the python code equivalent, no other commentary.'},
    {'role':'user','content':'Return the python code for: What is 123456 times 456789?, Dnt add any extra strung'}
    ],
    temperature = 0

)
response.choices[0].message.content

'123456 * 456789'

In [14]:
# We could then just run the code!
eval(response.choices[0].message.content)

56393342784

# Assistant with Code Interpreter

Clearly, we can expand on this idea! Here we show just using the LLM to convert word problems to simple Python math formulas, but in theory the LLMs can turn any natural language word description into Python code and then simply run the code itself! This is the key idea behind the code interpreter! OpenAI Assistants allow us to connect the LLM to the code-interpreter tool. Let's explore this further!


When creating an Assistant, you'll need to consider several parameters:

* Instructions: Here, you can define how you want the Assistant to behave or respond. These instructions guide the Assistant's overall tone, style, and the type of responses it should provide.

* Model Selection: You have the option to select from various GPT-3.5 or GPT-4 models, including those that are fine-tuned for specific tasks. If you're planning to use tools like the Retrieval tool, you'll need specific models like gpt-3.5-turbo-1106 or gpt-4-1106-preview.

* Tools Integration: The API supports integration with tools such as the Code Interpreter and Retrieval, both of which are developed and maintained by OpenAI. These tools enhance the capabilities of your Assistant by allowing it to interpret code or retrieve information more efficiently.

* Custom Functions: Another powerful feature of the API is the ability to define custom function signatures. This feature mirrors the function calling feature of the API, allowing your Assistant to perform specific tasks or operations based on your requirements.

Let's begin with a very simple assistant that has access to the code interpreter tool:

In [15]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You convert math problems into Python code and then run the code to show the answer",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo" # Check the docs for the latest models
)

### Threads

#### Creating a Thread

Let's break down the concept of creating a Thread in the context of using the Assistants API with OpenAI, focusing on making it more understandable and educational.

#### Understanding a "Thread"

Think of a Thread as a digital conversation or chat room. In real life, when you start talking to someone, you have a series of exchanges that make up a conversation. Similarly, in the digital world, when a user begins interacting with an API (like the OpenAI Assistants API), this interaction is grouped into what we call a "Thread."

#### Why Create a Thread?

1. **Organization**: Each user gets their own "Thread," like a personalized chat room. This keeps conversations organized and separate from each other.
2. **Context Management**: Within this Thread, you can pass along specific information or files that are relevant to that particular user. It's like having a personal folder for each user's conversation, keeping all their details and history in one place.

#### How Threads Work

- **No Size Limit**: You can think of a Thread as a never-ending conversation. There's no limit to how much you can add to it. You can continue the conversation, adding as many messages (or interactions) as needed.
- **Optimizing Conversations**: The Assistant (in this case, the AI model you're interacting with) is pretty smart. It manages the conversation efficiently by using special techniques like truncation. This means if the conversation gets too long, the Assistant will focus on the most relevant parts to keep the interaction smooth.
- **Simplifying Complexity**: When you use this system, you're essentially handing over the reins to the Assistant. It decides how much information (or how many input tokens) to consider for each response. This makes things easier for you because you don't have to worry about the technical aspects of managing the conversation's flow.

#### What Does This Mean for You?

- **Ease of Use**: You focus on what you want to ask or say, and the system handles the rest. It's designed to keep the conversation flowing without you having to manage the technical details.
- **Cost Consideration**: While this simplifies things greatly, it also means you have less direct control over potential costs since the Assistant decides how much data to process with each interaction.

In summary, creating a Thread when using the Assistants API is like starting a digital conversation with the AI, where each user gets their own personalized chat room. This system makes managing interactions simpler and more efficient, although it does mean you have less direct control over the conversation's technical aspects.

In [16]:
#Create Thread
thread = client.beta.threads.create()

In [18]:
#Add message to thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is 123456 times 456789?"
)
message

Message(id='msg_DdAor7kNTvCiUxkL00BxCEG0', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is 123456 times 456789?'), type='text')], created_at=1735951104, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_uevndJhV8BzjK8I6ByWSgINT')

In [20]:
#List mesages in thread
client.beta.threads.messages.list(thread_id='thread_uevndJhV8BzjK8I6ByWSgINT')

SyncCursorPage[Message](data=[Message(id='msg_DdAor7kNTvCiUxkL00BxCEG0', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is 123456 times 456789?'), type='text')], created_at=1735951104, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_uevndJhV8BzjK8I6ByWSgINT'), Message(id='msg_Oz0p6tuxmVnkKb3BkhkVsPWV', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is 123456 times 456789?'), type='text')], created_at=1735951093, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_uevndJhV8BzjK8I6ByWSgINT')], object='list', first_id='msg_DdAor7kNTvCiUxkL00BxCEG0', last_id='msg_Oz0p6tuxmVnkKb3BkhkVsPWV', has_more=False)

## Running the Assistant

### What Does "Run the Assistant" Mean?

Imagine the Assistant (like ChatGPT) as a helpful robot that's ready to answer questions or perform tasks. When you "Run the Assistant," you're basically waking up this robot and saying, "Hey, I have something for you to do."

### How Does It Work?

1. **Starting the Process**: You begin by creating what's called a "Run." This is like pressing the 'start' button on a machine. It tells the Assistant to start paying attention to the conversation, or "Thread," that's been going on.

2. **Reading the Thread**: Once the Run is initiated, the Assistant starts reading through the Thread. It's like giving the robot a history of what's been discussed so far, so it can understand the context and what needs to be answered or done next.

3. **Responding to Messages**: As the Run progresses, the Assistant adds its responses to the Thread. These are the messages with the role "assistant," showing that they are responses from the AI, not the user.

4. **Smart Context Management**: The Assistant is designed to automatically pick which previous messages it should consider while responding. This is important because it needs to know what's relevant to the current question or task. Think of it as the Assistant taking quick notes of the most important parts of the conversation to stay on track.

### What Should You Know?

- **Impact on Pricing and Performance**: The way the Assistant picks the context (or decides what parts of the conversation to focus on) affects two things: how much it costs to run the Assistant (since processing lots of data can be more expensive) and how well the Assistant performs (since focusing on the right context can lead to better, more accurate responses).
- **Optimization and Evolution**: This whole process has been fine-tuned based on OpenAI's experience with building systems like ChatGPT. However, it's always evolving. What this means is that the way the Assistant works today might be improved or changed slightly in the future to make it even better.

### Optional Instructions

- **Customizing the Run**: You can give the Assistant special instructions for each Run. This is like telling the robot, "This time, focus on this specific task, or answer in this particular way." But remember, these instructions will override the Assistant's default way of responding, so use this feature thoughtfully.

In summary, "Running the Assistant" is the step where you activate the AI to start processing and responding to the user's messages in a conversation. It involves reading the Thread, responding appropriately, and managing the conversation's context in a way that's optimized for cost and performance. This process is constantly evolving and can be customized with specific instructions for each Run.

In [21]:
#Create Run by adding threadid and assistant id
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Be very polite in your reply adn acknowledge that doing math is fun."
)

In [24]:
#Execute Run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [25]:
run.status

'completed'

In [26]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [27]:
messages

SyncCursorPage[Message](data=[Message(id='msg_RYVUoqy79vTqkATJ5L94izYJ', assistant_id='asst_V9wsi4kxGK6lHARi9K9Bv6eU', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="The product of 123456 and 456789 is 56,393,342,784. Math can be fun, don't you think?"), type='text')], created_at=1735951399, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_3a3wbIvxXZ9jz1SVDfq8Zqtr', status=None, thread_id='thread_uevndJhV8BzjK8I6ByWSgINT'), Message(id='msg_DdAor7kNTvCiUxkL00BxCEG0', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is 123456 times 456789?'), type='text')], created_at=1735951104, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_uevndJhV8BzjK8I6ByWSgINT'), Message(id='msg_Oz0p6tuxmVnkKb3BkhkVsPWV', assistant_id=None, attac

In [28]:
messages.data[0].content[0].text.value

"The product of 123456 and 456789 is 56,393,342,784. Math can be fun, don't you think?"

In [29]:
messages.data

[Message(id='msg_RYVUoqy79vTqkATJ5L94izYJ', assistant_id='asst_V9wsi4kxGK6lHARi9K9Bv6eU', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="The product of 123456 and 456789 is 56,393,342,784. Math can be fun, don't you think?"), type='text')], created_at=1735951399, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_3a3wbIvxXZ9jz1SVDfq8Zqtr', status=None, thread_id='thread_uevndJhV8BzjK8I6ByWSgINT'),
 Message(id='msg_DdAor7kNTvCiUxkL00BxCEG0', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is 123456 times 456789?'), type='text')], created_at=1735951104, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_uevndJhV8BzjK8I6ByWSgINT'),
 Message(id='msg_Oz0p6tuxmVnkKb3BkhkVsPWV', assistant_id=None, attachments=[], completed_at=Non

In [30]:
# This shows it in order of latest message
for thread_message in messages.data:
    print(thread_message.content[0].text.value)

The product of 123456 and 456789 is 56,393,342,784. Math can be fun, don't you think?
What is 123456 times 456789?
What is 123456 times 456789?


In [31]:
# This will show it in order of request
for thread_message in messages.data[::-1]:
    print(thread_message.content[0].text.value)
    print('\n')

What is 123456 times 456789?


What is 123456 times 456789?


The product of 123456 and 456789 is 56,393,342,784. Math can be fun, don't you think?




### Detailed Run Job Information:

We can actually see the steps the assistant took internally to create the reply:

In [32]:
from openai import OpenAI
client = OpenAI()

run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)
print(run_steps)


SyncCursorPage[RunStep](data=[RunStep(id='step_cM39teBNDLbLWnou20SrgJV5', assistant_id='asst_V9wsi4kxGK6lHARi9K9Bv6eU', cancelled_at=None, completed_at=1735951400, created_at=1735951399, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_3a3wbIvxXZ9jz1SVDfq8Zqtr', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_RYVUoqy79vTqkATJ5L94izYJ'), type='message_creation'), thread_id='thread_uevndJhV8BzjK8I6ByWSgINT', type='message_creation', usage=Usage(completion_tokens=32, prompt_tokens=182, total_tokens=214, prompt_token_details={'cached_tokens': 0}), expires_at=None), RunStep(id='step_3wznL0UNqIT1LsPPEG8Jkosv', assistant_id='asst_V9wsi4kxGK6lHARi9K9Bv6eU', cancelled_at=None, completed_at=1735951399, created_at=1735951397, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_3a3wbIvxXZ9jz1SVDfq8Zqtr', status='completed', step_detai

In [33]:
for step in run_steps:
    print(step.step_details)
    print('\n')

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_RYVUoqy79vTqkATJ5L94izYJ'), type='message_creation')


ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call_R4fwdzOKJnE5QrsHUCbuukHV', code_interpreter=CodeInterpreter(input='# Calculating the product of 123456 and 456789\nresult = 123456 * 456789\nresult', outputs=[]), type='code_interpreter')], type='tool_calls')




## Listing and Deleting Assistants

In [34]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)

[Assistant(id='asst_V9wsi4kxGK6lHARi9K9Bv6eU', created_at=1735950987, description=None, instructions='You convert math problems into Python code and then run the code to show the answer', metadata={}, model='gpt-3.5-turbo', name='Math Tutor', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0), Assistant(id='asst_tNFrui8AS1YyAKRdIhXfT6mJ', created_at=1735715415, description=None, instructions=None, metadata={}, model='gpt-4o', name=None, object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)]


In [36]:
my_assistants.data

[Assistant(id='asst_V9wsi4kxGK6lHARi9K9Bv6eU', created_at=1735950987, description=None, instructions='You convert math problems into Python code and then run the code to show the answer', metadata={}, model='gpt-3.5-turbo', name='Math Tutor', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0),
 Assistant(id='asst_tNFrui8AS1YyAKRdIhXfT6mJ', created_at=1735715415, description=None, instructions=None, metadata={}, model='gpt-4o', name=None, object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)]

In [38]:
assistant.id

'asst_V9wsi4kxGK6lHARi9K9Bv6eU'

In [39]:
response = client.beta.assistants.delete('asst_V9wsi4kxGK6lHARi9K9Bv6eU')
print(response)

AssistantDeleted(id='asst_V9wsi4kxGK6lHARi9K9Bv6eU', deleted=True, object='assistant.deleted')


In [40]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)

[Assistant(id='asst_tNFrui8AS1YyAKRdIhXfT6mJ', created_at=1735715415, description=None, instructions=None, metadata={}, model='gpt-4o', name=None, object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)]


In [41]:
response = client.beta.assistants.delete('asst_tNFrui8AS1YyAKRdIhXfT6mJ')
print(response)

AssistantDeleted(id='asst_tNFrui8AS1YyAKRdIhXfT6mJ', deleted=True, object='assistant.deleted')


In [42]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)

[]
